<a href="https://colab.research.google.com/github/taypa/testTaskMindBox/blob/main/test_task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.2/204.2 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 14.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612223 sha256=3a908879aec2aaae441842354399af6df7badbda543b337805b9d414d77dd429
  Stored in directory: /root/.cache/pip/wheels/19/b0/c8/6cb894117070e130fc44352c2a13f15b6c27e440d04a84fb48
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [6]:
products = spark.createDataFrame([
    (1, "Product A"),
    (2, "Product B"),
    (3, "Product C"),
    (4, "Product D"),
    (5, "Product E"),
    (6, "Product F")
], ["id", "product_name"])

categories = spark.createDataFrame([
    (1, "Category 1"),
    (2, "Category 2"),
    (3, "Category 3")
], ["id", "category_name"])

product_category = spark.createDataFrame([
    (1, 1),
    (1, 2),
    (2, 1),
    (3, 2),
    (3, 3),
    (4, 1)
], ["product_id", "category_id"])

In [25]:
from pyspark.sql.functions import col, when
def get_pairs(products, categories, product_category):
    #Соединяем датафреймы
    join_table = products.join(product_category, products.id == product_category.product_id,"left")
    join_table = join_table.join(categories, join_table.category_id == categories.id,"left")
    #Заменяем null на пустые строки
    join_table = join_table.withColumn("category_name", when(col("category_id").isNotNull(), col("category_name")).otherwise(""))
    #Удаляем лишние столбцы
    join_table = join_table.drop('id', 'product_id', 'category_id')
    return join_table

res = get_pairs(products, categories, product_category)
res.show()


+------------+-------------+
|product_name|category_name|
+------------+-------------+
|   Product F|             |
|   Product E|             |
|   Product A|   Category 1|
|   Product B|   Category 1|
|   Product D|   Category 1|
|   Product C|   Category 3|
|   Product A|   Category 2|
|   Product C|   Category 2|
+------------+-------------+

